In [ ]:
# Adapted from Robert Guthrie https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html
# And: https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
import sklearn
from sklearn.linear_model import LinearRegression
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import json
import zoib

torch.manual_seed(1)

In [ ]:
def split_train_test_val(df):
    ind_year = np.where(np.array(traj.index.names)=='year')[0][0]
    train_df = df.loc[df.index.get_level_values(ind_year)<=2010]
    val_df = df.loc[(df.index.get_level_values(ind_year)>2010) & (df.index.get_level_values(ind_year)<=2014)]
    test_df = df.loc[df.index.get_level_values(ind_year)>2014]
    return train_df, val_df, test_df

In [ ]:
def prep_lc_frac_df(ids=[]):
    """LC Frac csv is hardcoded! Change if you need it"""
    lc_df = pd.read_csv('../data/fraster_landcover_allyears_bigger.csv').set_index('id')
    if len(ids)>0:
        lc_df = lc_df.loc[ids]
    lc_frac = pd.DataFrame()
    for col in lc_df.columns:
        year = int(col[0:4])
        jsond = lc_df[col].str.replace(r'([0-9]+)(:)', r'"\1"\2', regex=True).apply(json.loads)
        temp_frac_df = (pd.json_normalize(jsond)/5000)
        temp_frac_df.columns = ['lcf{}'.format(lc) for lc in temp_frac_df.columns]
        temp_frac_df = temp_frac_df.assign(id=lc_df.index, year=year)
        lc_frac = lc_frac.append(temp_frac_df)
    lc_frac.fillna(0,inplace=True)
    
    return lc_frac.set_index(['id','year'])

In [ ]:
def read_join_csv(inun_csv, drop_zeros=True):
    # Prep inundation data
    inun_df = pd.read_csv(inun_csv)
    inun_df.set_index(['id','year','month'], inplace=True)
    inun_df = inun_df.loc[~inun_df['inundation'].isna()]
    if drop_zeros:
        max_inun = inun_df.groupby('id').agg({'inundation':'max'})
        zero_ids = max_inun.loc[max_inun['inundation']==0].index
        inun_df.drop(zero_ids, inplace=True)
        if inun_df.shape[0]==0:
            return 
        
    # Prep weather data
    weather_csv = inun_csv.replace('inun_frac_','weather_')
    weather_df = pd.read_csv(weather_csv)
    weather_df.set_index(['id','year','month'], inplace=True)
    joined_df = weather_df.join(inun_df, how='inner')
    
    # Finally, prep landcover fraction dataframe
    # Both prep and join are a bit slow
    # Could prep into fractions ahead of time
    # And also split up lc df by county
    lc_frac_df = prep_lc_frac_df(ids=joined_df.index.get_level_values(0).unique())
    joined_df = joined_df.join(lc_frac_df, how='inner')
    
    return joined_df

# Load data

In [ ]:
target_num_playas = 1

In [ ]:
inun_csv_list = glob.glob('../data/state_county_csvs/counties/inun_frac*')

In [ ]:
joined_df = pd.DataFrame()
while joined_df.index.get_level_values(0).unique().shape[0] <= target_num_playas:
    rand_csv = np.random.choice(inun_csv_list)
    joined_df = pd.concat([joined_df, read_join_csv(rand_csv, drop_zeros=True)])
    
joined_df.fillna(0, inplace=True)

joined_df = joined_df.loc[joined_df.index.get_level_values(0).unique()[:target_num_playas]]

# Prep Data

In [ ]:
traj = joined_df
traj = traj.drop(columns=['area'])#[['inundation', 'acres', 'vpd', 'temp','precip']]
n_features = traj.shape[1]
traj['inundation'].plot()

In [ ]:
len_of_timeseries = 418
new_ids = np.array([
    np.repeat(i, len_of_timeseries) for i in range(int(traj.shape[0]/len_of_timeseries))]
).flatten()
traj = traj.assign(id=new_ids) # Put id at end for embedding

In [ ]:
# Pop inundation to end
inun = traj.pop('inundation')
traj['inundation'] = inun


# Prep and run model

In [ ]:
# Params to set
hidden_dim = 128
embedding_dim = 1

num_layers=1
learning_rate = 0.05
num_epochs = 2000
weight_decay = 0
lr_gamma = 0.95
lr_decay_step_size =25
regularization_weight = 0
batch_size = 1

early_stopping=25

In [ ]:
def split_by_id(x, y):
    seq_length = np.sum((x[:,-1]==x[0,-1]))
    seq_starts = np.arange(0, x.shape[0], seq_length)
    x_arr = np.array([np.array(x[i:(i+seq_length)]) for i in seq_starts])
    y_arr = np.array([np.array(y[i:(i+seq_length)]) for i in seq_starts])
    return x_arr, y_arr

In [ ]:
scaler = StandardScaler()
train, val, test = split_train_test_val(traj)
train_X, train_y = train.values[:, :-1], train.values[:, -1]
val_X, val_y = val.values[:, :-1], val.values[:, -1]
test_X, test_y = test.values[:, :-1], test.values[:, -1]
# Run scaler, but not on ID
train_X[:,:-1] = scaler.fit_transform(train_X[:,:-1])
val_X[:,:-1] = scaler.transform(val_X[:,:-1])
test_X[:,:-1] = scaler.transform(test_X[:,:-1])

num_playas = int(train_X[:, -1].max())+1
lstm_input_size = traj.shape[1]-1

In [ ]:
# Create train, val, and test sets
train_X_array, train_y = split_by_id(train_X, train_y)
val_X_array, val_y = split_by_id(val_X, val_y)
test_X_array, test_y = split_by_id(test_X, test_y)


In [ ]:
train_ds = torch.utils.data.TensorDataset(torch.Tensor(train_X_array), torch.Tensor(train_y),
                                          torch.Tensor(val_X_array), torch.Tensor(val_y))
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)

In [ ]:
# Here we define our model as a class
class LSTM(nn.Module):

    def __init__(self, input_dim, embedding_dim, num_playas, hidden_dim, batch_size, output_dim=1,
                    num_layers=1):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.num_playas = num_playas

        # Define embedding layer
        self.embedding = nn.Embedding(self.num_playas, self.embedding_dim)
        
        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim + self.embedding_dim - 1, self.hidden_dim, self.num_layers, batch_first=True)

        # Define activations for output
        self.linear = nn.Linear(self.hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.exp = torch.exp
        self.sigmoid = nn.Sigmoid()
    

    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of input: [batch_size, timesteps, input_dims]
        # shape of lstm_out: [batch_size, timesteps, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        # Shape of y_pred: [batch_size, timesteps, 4]

        # Run ids through embedding layer
        self.emb_layer = self.embedding(input[:,:,-1].long())
        
        # Concat and run through LSTM
        lstm_out, self.hidden = self.lstm(torch.cat((input[:,:,:-1],self.emb_layer), 2))

        
        # Run activation and get outputs
        lin_act = self.linear(lstm_out)
        y_pred = torch.cat((self.sigmoid(lin_act[:,:,0:2]),torch.exp(lin_act[:,:,2:4])), 2)

        return y_pred


model = LSTM(input_dim = lstm_input_size,
             embedding_dim=embedding_dim,
             num_playas=num_playas,
             hidden_dim=hidden_dim,
             batch_size=batch_size,
             output_dim=4,
             num_layers=num_layers)

In [ ]:
l1_loss = nn.L1Loss()
    
optimiser = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimiser, step_size=lr_decay_step_size, gamma=lr_gamma)
#####################---------------------------------------------------------------------------
# Train model
#####################

loss_history = []
val_loss_history = []

best_loss = 1000
for t in range(num_epochs):
    epoch_loss = 0
    val_epoch_loss = 0
    total_items = 0
    val_total_items = 0
    # List to store all predictions
    all_train_pred = []
    all_val_pred = []
    for x_batch, y_batch, val_x_batch, val_y_batch in train_loader: 
        
        # Clear stored gradient
        model.zero_grad()

        # Initialise hidden state
        model.hidden = model.init_hidden()

        
        # Training: Predict and calc loss
        train_pred = model(x_batch)
        # Flatten to (batch_size*timesteps) x 4
        train_pred = train_pred.view(train_pred.shape[0]*train_pred.shape[1], 4) 
        all_train_pred.append(train_pred)
        loss = zoib.zoib_loss(
            train_pred,
            y_batch.view(y_batch.shape[0]*y_batch.shape[1], 1)[:,0] # Flatten to (batch_size*timesteps) X 1
        ).float()
        
        # Tracking mean loss across batches
        epoch_loss += train_pred.shape[0]*loss.item()
        total_items += train_pred.shape[0]
        
        
        # Validation: predict and calc loss
        val_pred = model(val_x_batch)
        # Flatten to (batch_size*timesteps) x 4
        val_pred = val_pred.view(val_pred.shape[0]*val_pred.shape[1], 4)
        all_val_pred.append(val_pred)
        val_loss = zoib.zoib_loss(
            val_pred, 
            val_y_batch.view(val_y_batch.shape[0]*val_y_batch.shape[1], 1)[:,0]  # Flatten to (batch_size*timesteps) X 1
        ).float()

        # Tracking mean loss across batches
        val_epoch_loss += val_pred.shape[0]*val_loss.item()
        val_total_items += val_pred.shape[0]

        
        # Zero out gradient
        optimiser.zero_grad()

        # Backward pass
        loss.backward()

        # Update parameters
        optimiser.step()

    # LR decay
    scheduler.step()
    
    epoch_loss = epoch_loss/total_items
    val_epoch_loss = val_epoch_loss/val_total_items
    loss_history.append(epoch_loss)
    val_loss_history.append(val_epoch_loss)
    if t%10==0:
        print("Epoch ", t, "Train Loss: ", epoch_loss, ", Val Loss: ", val_epoch_loss, ", LR: ", optimiser.param_groups[0]["lr"])
        
    if np.isnan(epoch_loss):
        break
        
    # Early stopping
    if epoch_loss < best_loss:
        i = 0
        # Save best loss, predictoins, and hidden state
        best_loss = epoch_loss
        best_train_pred = torch.cat(all_train_pred, dim=0)
        best_val_pred = torch.cat(all_val_pred, dim=0)
        best_hidden_1, best_hidden_2 = model.hidden[0].clone().detach().numpy(), model.hidden[1].clone().detach().numpy()
    
    if (i > early_stopping):
        break

# View results

In [ ]:
plt.plot(loss_history)

In [ ]:
def zoib_expected(t):
    # E = q*(1-p) + (1-p-q)*(conc1/(conc1+conc0))
    # Or # = prob_1_given_not0*(1-prob_0) + (1 - prob_bernoulli)*(expect_val_beta)
    t = t.detach().numpy()
    prob_1 = t[:,1]*(1-t[:,0])
    prob_beta = (1 - t[:,0])*(1 - t[:,1])
    beta_expected = t[:,2]/(t[:,3]+t[:,2])
    return prob_1 + prob_beta*beta_expected

In [ ]:
plt.scatter(zoib_expected(best_train_pred), train_y.flatten())

In [ ]:
pd.DataFrame({'Pred':zoib_expected(best_train_pred), 'True':train_y.flatten()}).plot(xlim=[0,120])

In [ ]:
plt.scatter(zoib_expected(best_val_pred), val_y.flatten())

In [ ]:
pd.DataFrame({'Pred':zoib_expected(best_val_pred), 'True':val_y.flatten()}).plot()

In [ ]:
param_df = pd.DataFrame(best_train_pred.detach().numpy())
param_df.columns = ['p','q','conc1','conc0']
pd.plotting.scatter_matrix(param_df)
plt.show()